# Deep learning 1

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('../lxmls-toolkit')
import lxmls
import scipy
import numpy as np

## Exercise 5.1

Go to **```lxmls/deep\_learning/mlp.py:class NumpyMLP```**  and check the function **```grads()```** and complete the code of the NumpyMLP class with the Backpropagation recursion that we just saw.
Once you are done. Try different network geometries by increasing the number of
layers and layer sizes e.g.

In [167]:
#load data
import numpy as np
import lxmls.readers.sentiment_reader as srs
scr = srs.SentimentCorpus("books")

train_x = scr.train_X.T
train_y = scr.train_y[:, 0]
test_x = scr.test_X.T
test_y = scr.test_y[:, 0]

2000
1600


In [168]:
# Load mlp and sgd from toolit
from lxmls import deep_learning 
from lxmls.deep_learning import mlp
import lxmls.deep_learning.sgd as sgd


In [169]:
train_x.shape, train_y.shape

((13989, 1600), (1600,))

In [171]:
# Model parameters
geometry = [train_x.shape[0], 100,100, 2]
actvfunc = ['sigmoid', 'sigmoid','softmax'] 
# Instantiate model
mlp      = mlp.NumpyMLP(geometry, actvfunc) 

In [172]:
# Model parameters
n_iter = 5
bsize  = 5
lrate  = 0.01

In [173]:
# Train
sgd.SGD_train(mlp, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y))
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test  = sgd.class_acc(mlp.forward(test_x), test_y)[0]
print "MLP (%s) Amazon Sentiment Accuracy train: %f test: %f" % (geometry, acc_train,acc_test)

Batch 321/321 (100%)   Epoch  1/ 5 in 16.31 seg
Batch 321/321 (100%)   Epoch  2/ 5 in 16.20 seg
Batch 321/321 (100%)   Epoch  3/ 5 in 16.47 seg
Batch 321/321 (100%)   Epoch  4/ 5 in 16.54 seg
Batch 321/321 (100%)   Epoch  5/ 5 in 16.63 seg

MLP ([13989, 100, 100, 2]) Amazon Sentiment Accuracy train: 0.945000 test: 0.797500


In [176]:
# Load mlp and sgd from toolit
from lxmls import deep_learning 
from lxmls.deep_learning import mlp
import lxmls.deep_learning.sgd as sgd

# Model parameters
geometry = [train_x.shape[0], 300, 2]
actvfunc = ['sigmoid', 'softmax'] 
# Instantiate model
mlp      = mlp.NumpyMLP(geometry, actvfunc) 

In [177]:
# Train
sgd.SGD_train(mlp, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y))
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test  = sgd.class_acc(mlp.forward(test_x), test_y)[0]
print "MLP (%s) Amazon Sentiment Accuracy train: %f test: %f" % (geometry, acc_train,acc_test)

Batch 321/321 (100%)   Epoch  1/ 5 in 48.97 seg
Batch 321/321 (100%)   Epoch  2/ 5 in 48.05 seg
Batch 321/321 (100%)   Epoch  3/ 5 in 50.11 seg
Batch 321/321 (100%)   Epoch  4/ 5 in 49.49 seg
Batch 321/321 (100%)   Epoch  5/ 5 in 49.34 seg

MLP ([13989, 300, 2]) Amazon Sentiment Accuracy train: 0.960000 test: 0.790000


## Exercise 5.2 
#### Begin Exercise 5.2

Get in contact with Theano. Learn the difference between a symbolic
representation and a function. Start by implementing the first layer of our
previous MLP in Numpy 

In [178]:
# Numpy code
x        = test_x             # Test set 
W1, b1   = mlp.params[:2]     # Weights and bias of fist layer 
z1       = np.dot(W1, x) + b1 # Linear transformation
tilde_z1 = 1/(1+np.exp(-z1))  # Non-linear transformation  

In [180]:
# Theano code. 
# NOTE: We use undescore to denote symbolic equivalents to Numpy variables. 
# This is no Python convention!.
import theano
import theano.tensor as T
_x = T.matrix('x')

Note that this variable does not have any particular value, nor a space
reserved in memory for it. It contains just a symbolic definition of what the
variable can contain. The particular values will be given when we use it to
compile a function. 

We could actually use the same definition format to define the weights and give
their particular values as inputs to the compiled function. However, since we
will be using a more complicated format in later exercises, we will use it here
as well. The **```shared```** class allows to define variables that are shared
across functions. They are also given a concrete value so that we do not need
to give it for each function call. This format is therefore ideal for the
weights of our network.

In [181]:
_W1 = theano.shared(value=W1, name='W1', borrow=True) 
_b1 = theano.shared(value=b1, name='b1', borrow=True, broadcastable=(False, True)) 

Now lets describe the operations we want to do with the variables. Again only
symbolically. This is done by replacing our usual operations by Theano symbolic
ones when necessary e. g. the internal product dot() or the sigmoid. Some
operations like e.g. $+$ are automatically recognized by Theano (operator
overloading). 

In [182]:
_z1            = T.dot(_W1, _x) + _b1
_tilde_z1      = T.nnet.sigmoid(_z1)
# Keep in mind that naming variables is useful when debugging
_z1.name       = 'z1'
_tilde_z1.name = 'tilde_z1'

When debugging the code it is often useful to print the graph of computations.

In [184]:
# Perceptron computation graph
theano.printing.debugprint(_tilde_z1)


sigmoid [@A] 'tilde_z1'   
 |Elemwise{add,no_inplace} [@B] 'z1'   
   |dot [@C] ''   
   | |W1 [@D]
   | |x [@E]
   |b1 [@F]


It is important to keep in mind that, until this point, we do not have a
function we can use to produce any practical input. In order to obtain this we
have to compile this function by calling    

In [185]:
layer1 = theano.function([_x], _tilde_z1)

Note the use of [ ] for the input variables, even if we just specify one
variable. We can now do a test to compare the Numpy and Theano implementations
and see that they give the same outputs.

In [187]:
# Check Numpy and Theano match
if np.allclose(tilde_z1, layer1(x.astype(theano.config.floatX))):
    print "\nNumpy and Theano Perceptrons are equivalent"
else:
    raise ValueError, "Numpy and Theano Perceptrons are different"


Numpy and Theano Perceptrons are equivalent


#### End exercise 5.2

#### Symbolic forward pass
In the previous section you have seen how to create symbolic Theano functions
with shared parameters. You have thus all you need to implement the whole
forward pass of a generic MLP in Theano.


## Exercise 5.3

#### Begin Exercise 5.3

Complete the method **```_forward()```** inside of the **```lxmls/deep\_learning/mlp.py```**, in the class TheanoMLP.

Note that this is called only once at the initialization of the
class. To debug your implementation put a breakpoint at the \_\_init\_\_
function call. Hint: Note that this is very similar to NumpyMLP.forward().
You just need to keep track of the symbolic variable representing the output of
the network after each layer is applied and compile the function at the end.
After you are finished instantiate a Theano class and check that Numpy and
Theano forward pass are the same. 


In [197]:
from lxmls.deep_learning import mlp 

In [199]:
mlp_a = mlp.NumpyMLP(geometry, actvfunc)
mlp_b = mlp.TheanoMLP(geometry, actvfunc)

#### End Exercise 5.3


## Symbolic differentiation
In the previous section we compiled the forward pass of a MLP. In this section
we will do the same with the cost used for training. We will also derive the
gradients although this will be trivial once we have the cost function compiled.     

## Exercise 5.4

We first see an example that does not use any of the code in TheanoMLP but
rather continues from what you wrote in exercise 6.3. In this exercise you
completed a sigmoid layer with Theano. To get some values for the weights we
used the first layer of the network you trained in 6.2. now we are going to use
the second layer as well. This is thus assuming that your network in 6.2 has
only two layers e.g. the recommended geometry (I, 20, 2). Make sure this is the
case before starting this exercise.  

For the sake of clarity, lets write here the part of Ex. 5.2 that we had completed

In [203]:
# Get the values from our MLP from Ex 6.2
W1, b1   = mlp_a.params[:2]     # Weights and bias of fist layer 
# First layer symbolic variables
_x  = T.matrix('x')
_W1 = theano.shared(value=W1, name='W1', borrow=True) 
_b1 = theano.shared(value=b1, name='b1', borrow=True, broadcastable=(False, True)) 
# First layer symbolic expressions
_z1       = T.dot(_W1, _x) + _b1
_tilde_z1 = T.nnet.sigmoid(_z1)

## Example XOR

In [7]:
# load data
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

Y = np.array([0,1,1,0])

In [67]:
W_1_shape = (2,3)
W_2_shape = (3,2)

W_1 = np.random.normal(loc=0,scale=0.1, size=W_1_shape)
b_1 = np.zeros(3)

W_2 = np.random.normal(loc=0,scale=0.1, size=W_2_shape)
b_2 = np.zeros(2)

weights = [W_1, W_2]
biases = [b_1,b_2]

In [ ]:
class activation_layer(object):
    

In [42]:
def relu(z):
    z = np.array(z)
    return z*(z>0)

In [43]:
relu([5,-2])

array([5, 0])

In [54]:
def softmax (z):
    return np.exp(z)/np.sum(np.exp(z))

In [61]:
softmax(np.array([5,1]))

array([ 0.98201379,  0.01798621])

In [66]:
activations = [relu, softmax]

In [59]:

class CrossEntropy(object):

    def __init__(self):
        print("\nTraining cost: CrossEntropy ")

    def compute_cost(self, pred, target):
        """
        Return the cost associated for "Xpred" and desired output "Xtarget".
        Note that the function is vectorized and
           - rows in pred are training examples. 
           - rows in target are output predictions

        """
        return np.mean((target - pred)**2)

    def delta(self, preactivation, activation, target):
        """
        Return the error delta from the output layer.
        """
        # nablaC_activation = (activation-target)
        delta = 0.5 * (activation - target) #* sigmoid_prime(preactivation)
        return delta


In [62]:
mse = MSE()


Training cost: MSE 


In [ ]:
writs

In [56]:
def forward_propagate(x, weights, biases, activations):
    """
    Computes the activations of the first hidden layer and output layer.
    """
    activations = []
    
    z_1 = np.dot(x,W1) + b_1
    z_1_tilde = act1(z1)
    activations.append(z_1_tilde )
    
    z_2 = np.dot(z_1_tilde, W2) + b_2
    z_2_tilde =  act2(z_2)
    activations.append(z_2_tilde)
    
    return activations

In [ ]:
def compute_grad(activations, target, cost):
    """
    
    """
    output = activations[-1]
    